## Setup & Installation

In [12]:
%%capture --no-stderr
%pip install --quiet -U langchain-openai langchain-community langchain-core tavily-python pydantic python-docx docx2pdf reportlab

In [13]:
import os
import getpass
from typing import List, Dict, Any, Set
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Markdown, display
import json

## API Keys Configuration

In [14]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

print("API keys configured!")

API keys configured!


## Data Models

In [15]:
class Activity(BaseModel):
    """Single activity in the itinerary"""
    name: str = Field(description="Activity name")
    description: str = Field(description="Brief description")
    location: str = Field(description="Specific location")
    duration_hours: float = Field(description="Duration in hours")
    estimated_cost: float = Field(description="Estimated cost in USD")
    time_of_day: str = Field(description="Morning/Afternoon/Evening")
    category: str = Field(description="Food/Sightseeing/Activity/Transport")

class DayPlan(BaseModel):
    """Plan for a single day"""
    day_number: int
    activities: List[Activity]
    total_cost: float

class ConflictResolution(BaseModel):
    """Instructions for resolving conflicts in the itinerary"""
    has_conflicts: bool = Field(description="Whether conflicts were found")
    issues_found: List[str] = Field(description="List of issues identified")
    resolutions: List[str] = Field(description="Actions to fix the issues")

print("Data models defined!")

Data models defined!


## Initialize AI Models

In [16]:
# Initialize ChatGPT
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Initialize search tool
tavily_search = TavilySearchResults(max_results=5)

print("AI models initialized!")

AI models initialized!


## Helper Functions

In [17]:
def query_destination_guide(destination: str, budget_per_day: float) -> str:
    """Query destination information"""
    query = f"{destination} travel guide budget ${budget_per_day:.0f} per day attractions"
    try:
        results = tavily_search.invoke(query)
        return "\n\n".join([f"Source: {doc['url']}\n{doc['content']}" for doc in results])
    except Exception as e:
        print(f"Search failed: {e}")
        return f"General information about {destination}."

def get_visited_locations(daily_plans: List[DayPlan]) -> Set[str]:
    """Extract all unique locations visited so far"""
    locations = set()
    for day in daily_plans:
        for activity in day.activities:
            # Normalize location names for comparison
            loc = activity.location.lower().strip()
            locations.add(loc)
    return locations

def plan_single_day(destination: str, day_number: int, num_days: int,
                     daily_budget: float, guide: str,
                     visited_locations: Set[str] = None,
                     target_utilization: float = 0.92) -> DayPlan:
    """Plan activities for a single day avoiding previously visited locations"""
    if visited_locations is None:
        visited_locations = set()

    target_cost = daily_budget * target_utilization

    visited_list = "\n".join([f"- {loc}" for loc in visited_locations]) if visited_locations else "None yet"

    planning_prompt = f"""Plan Day {day_number} of a {num_days}-day trip to {destination}.

Daily Budget: ${daily_budget:.2f}
TARGET COST: ${target_cost:.2f} (aim to use 90-95% of budget)
Guide Info: {guide[:1000]}...

LOCATIONS ALREADY VISITED (DO NOT REPEAT):
{visited_list}

CRITICAL RULES:
1. DO NOT visit any location from the "already visited" list
2. Choose DIFFERENT attractions, restaurants, and areas
3. Each location must be UNIQUE - not visited on previous days
4. Total cost should be CLOSE TO ${target_cost:.2f} (90-95% of budget)

Create a realistic itinerary with:
- 4-6 activities (meals at DIFFERENT restaurants, NEW sightseeing spots, transport)
- UNIQUE locations not in the visited list
- Specific location names
- Logical morning to afternoon to evening flow
- Quality activities that maximize budget value
"""

    structured_llm = llm.with_structured_output(DayPlan)
    day_plan = structured_llm.invoke([
        SystemMessage(content=planning_prompt),
        HumanMessage(content=f"Create day {day_number} plan with NEW locations only. Aim for ${target_cost:.2f}.")
    ])

    # Ensure budget utilization
    if day_plan.total_cost < daily_budget * 0.85:
        scale = target_cost / day_plan.total_cost if day_plan.total_cost > 0 else 1
        for activity in day_plan.activities:
            activity.estimated_cost *= scale
        day_plan.total_cost = min(target_cost, daily_budget)

    if day_plan.total_cost > daily_budget:
        scale = daily_budget / day_plan.total_cost
        for activity in day_plan.activities:
            activity.estimated_cost *= scale
        day_plan.total_cost = daily_budget

    return day_plan

def detect_conflicts(destination: str, num_days: int, total_budget: float,
                     daily_plans: List[DayPlan]) -> ConflictResolution:
    """Detect conflicts including duplicate locations"""
    total_spent = sum(day.total_cost for day in daily_plans)

    # Track locations per day for duplicate detection
    location_days = {}
    for day in daily_plans:
        for activity in day.activities:
            loc = activity.location.lower().strip()
            if loc not in location_days:
                location_days[loc] = []
            location_days[loc].append(day.day_number)

    # Find duplicates
    duplicate_locations = {loc: days for loc, days in location_days.items() if len(days) > 1}

    itinerary_summary = "\n\n".join([
        f"Day {day.day_number} (${day.total_cost:.2f}):\n" +
        "\n".join([f"  {a.time_of_day}: {a.name} at {a.location} ({a.duration_hours}h)"
                   for a in day.activities])
        for day in daily_plans
    ])

    duplicate_info = ""
    if duplicate_locations:
        duplicate_info = "\n\nDUPLICATE LOCATIONS FOUND:\n" + "\n".join(
            [f"- '{loc}' visited on days: {days}" for loc, days in duplicate_locations.items()]
        )

    review_prompt = f"""Analyze this {num_days}-day {destination} itinerary:

{itinerary_summary}
{duplicate_info}

Budget: ${total_budget:.2f} | Spent: ${total_spent:.2f}

Identify conflicts:
1. DUPLICATE LOCATIONS - Same location visited multiple times (CRITICAL)
2. Time conflicts (overlapping activities, insufficient travel time)
3. Budget issues (over budget, unrealistic costs)
4. Logical issues (missing meals, poor pacing, activities at wrong time)
5. Location issues (too much travel, illogical routing)

For each issue, specify:
- What's wrong (include specific location names if duplicates)
- How to fix it (suggest alternative locations)
"""

    structured_llm = llm.with_structured_output(ConflictResolution)
    return structured_llm.invoke([
        SystemMessage(content="You are a travel planner identifying itinerary conflicts, especially duplicate locations."),
        HumanMessage(content=review_prompt)
    ])

def resolve_conflicts(destination: str, num_days: int, total_budget: float,
                      daily_plans: List[DayPlan], conflicts: ConflictResolution) -> List[DayPlan]:
    """Automatically resolve conflicts ensuring unique locations"""
    if not conflicts.has_conflicts:
        return daily_plans

    fixed_plans = []
    remaining_budget = total_budget
    target_utilization = 0.92

    for day_num in range(1, num_days + 1):
        remaining_days = num_days - day_num + 1
        daily_limit = remaining_budget / remaining_days
        target_cost = daily_limit * target_utilization

        # Get locations already used in fixed days
        used_locations = get_visited_locations(fixed_plans)
        used_list = "\n".join([f"- {loc}" for loc in used_locations]) if used_locations else "None yet"

        original_day = daily_plans[day_num - 1]

        fix_prompt = f"""Fix Day {day_num} of a {num_days}-day trip to {destination}.

LOCATIONS ALREADY USED IN OTHER DAYS (AVOID THESE):
{used_list}

ISSUES TO RESOLVE:
{chr(10).join(f'- {issue}' for issue in conflicts.issues_found if f'Day {day_num}' in issue or 'duplicate' in issue.lower())}

RESOLUTIONS NEEDED:
{chr(10).join(f'- {res}' for res in conflicts.resolutions)}

ORIGINAL DAY {day_num} ACTIVITIES:
{chr(10).join(f'{a.time_of_day}: {a.name} at {a.location} (${a.estimated_cost:.2f})' for a in original_day.activities)}

Daily Budget: ${daily_limit:.2f}
TARGET: ${target_cost:.2f} (90-95% of budget)

CRITICAL REQUIREMENTS:
1. Use COMPLETELY DIFFERENT locations not in the "already used" list
2. Choose NEW restaurants, NEW attractions, NEW areas
3. Each location must be UNIQUE across all days
4. Resolve all timing and logical conflicts
5. Aim for ${target_cost:.2f} total cost
"""

        structured_llm = llm.with_structured_output(DayPlan)
        fixed_day = structured_llm.invoke([
            SystemMessage(content="You fix travel itinerary conflicts, especially duplicate locations."),
            HumanMessage(content=fix_prompt)
        ])

        # Budget adjustment
        if fixed_day.total_cost < daily_limit * 0.85:
            scale = target_cost / fixed_day.total_cost if fixed_day.total_cost > 0 else 1
            for activity in fixed_day.activities:
                activity.estimated_cost *= scale
            fixed_day.total_cost = min(target_cost, daily_limit)

        if fixed_day.total_cost > daily_limit:
            scale = daily_limit / fixed_day.total_cost
            for activity in fixed_day.activities:
                activity.estimated_cost *= scale
            fixed_day.total_cost = daily_limit

        fixed_plans.append(fixed_day)
        remaining_budget -= fixed_day.total_cost

    return fixed_plans

print("Helper functions defined!")

Helper functions defined!
